In [2]:
#Load the dataset
from datasets import load_dataset

import json

# Load the HateXplain dataset
dataset = load_dataset("../hatexplain/hatexplain.py")

# Inspect the dataset
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['id', 'annotators', 'rationales', 'post_tokens'],
        num_rows: 15383
    })
    validation: Dataset({
        features: ['id', 'annotators', 'rationales', 'post_tokens'],
        num_rows: 1922
    })
    test: Dataset({
        features: ['id', 'annotators', 'rationales', 'post_tokens'],
        num_rows: 1924
    })
})


In [3]:
train_data = dataset["train"]
val_data = dataset["validation"]
test_data = dataset["test"]

print(train_data[0])


{'id': '23107796_gab', 'annotators': {'label': [0, 2, 2], 'annotator_id': [203, 204, 233], 'target': [['Hindu', 'Islam'], ['Hindu', 'Islam'], ['Hindu', 'Islam', 'Other']]}, 'rationales': [[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 'post_tokens': ['u', 'really', 'think', 'i', 'would', 'not', 'have', 'been', 'raped', 'by', 'feral', 'hindu', 'or', 'muslim', 'back', 'in', 'india', 'or', 'bangladesh', 'and', 'a', 'neo', 'nazi', 'would', 'rape', 'me', 'as', 'well', 'just', 'to', 'see', 'me', 'cry']}


In [3]:
#Focus on train data for now
#Observe the train data

#Count the number of annotators
count_annotators ={}
count_mistmatch = 0
for data in train_data:
    count = len(data['annotators']['annotator_id'])
    if count !=len(data["annotators"]["label"]) or count !=count != len(data["annotators"]["target"]):
        count_mistmatch +=1
    if count in count_annotators:
        count_annotators[count] +=1
    else:
        count_annotators[count] = 1

print(count_annotators)
print("count_mistmatch", count_mistmatch)


{3: 15383}
count_mistmatch 0


In [5]:
#count an item where the number of annotators don't match the number of rationales
count_rationale_match_annotators = 0
count_rational_not_match_annotators = 0
count_rationales = {}
for data in train_data:
    count = len(data['rationales'])
    if count in count_rationales:
        count_rationales[count] += 1
    else:
        count_rationales[count] = 1
        
    if len(data['annotators']["label"]) == count:
        count_rationale_match_annotators += 1
    else:
        count_rational_not_match_annotators += 1
        
print("count_rationale_match_annotators: ", count_rationale_match_annotators)
print("count_rational_not_match_annotators", count_rational_not_match_annotators)
print(count_rationales)

count_rationale_match_annotators:  3518
count_rational_not_match_annotators 11865
{2: 5614, 3: 3518, 0: 6251}


In [15]:
#Are there data labeled as offensive or hate speech but no annotation?
no_rationale_data_list = []
for data in train_data:
    count = len(data["rationales"])
    if count == 0:
        no_rationale_data_list.append(data["annotators"]["label"])

no_rationale_normal = []
no_rationale_else = []
for data in no_rationale_data_list:
    if data[0] == data[1] == data[2] ==1: #normal
        no_rationale_normal.append(data)
    else:
        no_rationale_else.append(data)

print("no_rationale_normal (all three anotators thought the speech was normal)", len(no_rationale_normal))
print("no_rationale_else (not all through it was normal)", len(no_rationale_else))

no_rationale_abnormal = []
for data in no_rationale_else:
    count = 0
    for i in data:
        if i == 1:
            count +=1
    if count < 2:
        no_rationale_abnormal.append(data)
print("no_rationale_abnormal (the majority vote was not normal): ", len(no_rationale_abnormal))


no_rationale_normal (all three anotators thought the speech was normal) 4096
no_rationale_else (not all through it was normal) 2155
no_rationale_abnormal (the majority vote was not normal):  0


In [17]:
#check # of rationales vs # of # of hate speech labels
count_hate_label_match_rationales =0
count_hate_label_no_match_rationales =0
for data in train_data:
    count_hate_label = data["annotators"]["label"].count(0)
    count_rationales = len(data["rationales"])
    if count_hate_label == count_rationales:
        count_hate_label_match_rationales += 1
    else:
        count_hate_label_no_match_rationales += 1
        
print("count_hate_label_match_rationales: ", count_hate_label_match_rationales)
print("count_hate_label_no_match_rationales", count_hate_label_no_match_rationales)
    

count_hatel_label_match_rationales:  9016
count_hatel_label_no_match_rationales 6367


In [13]:
#Check target
target_if_normal = []
for data in train_data:
    labels = data["annotators"]["label"]
    if labels[0]==labels[1]==labels[2] == 1: #normal
        target_if_normal.append(data["annotators"]["target"])

print(target_if_normal[:100])

[[['Refugee'], ['Refugee'], ['None']], [['Arab', 'Caucasian'], ['None'], ['Arab']], [['Women'], ['None'], ['Women']], [['None'], ['None'], ['None']], [['None'], ['None'], ['None']], [['Islam', 'Women'], ['Arab', 'Islam', 'Women'], ['Arab', 'Islam']], [['None'], ['African'], ['None']], [['None'], ['None'], ['None']], [['None'], ['None'], ['None']], [['None'], ['None'], ['None']], [['None'], ['None'], ['None']], [['Other'], ['None'], ['Other']], [['None'], ['Women', 'Homosexual'], ['None']], [['None'], ['Other'], ['None']], [['None'], ['None'], ['None']], [['None'], ['None'], ['None']], [['None'], ['None'], ['None']], [['None'], ['None'], ['None']], [['None'], ['None'], ['None']], [['None'], ['Other'], ['None']], [['None'], ['None'], ['None']], [['None'], ['Women', 'None'], ['Other']], [['None'], ['None'], ['None']], [['None'], ['None'], ['None']], [['None'], ['Islam'], ['None']], [['None'], ['None'], ['None']], [['None'], ['None'], ['None']], [['None'], ['Caucasian'], ['None']], [['None

In [4]:
#Let's process the training/eval data
from collections import Counter

def process_easy_data(dataset):
    def get_majority_element(lst):
        # Count occurrences of each element
        count = Counter(lst)
        
        # Check if there is a majority (i.e., an element that appears at least twice)
        if count.most_common(1)[0][1] >= 2:
            return count.most_common(1)[0][0]
        else:
            return 3  # Indicate undecided if no majority
        
    easy_data =[]
    for data in dataset:
        labels = data["annotators"]['label']
        targets = data["annotators"]["target"]
        
        final_label = get_majority_element(labels)
        
        final_targets = []
        target_dicts = {}
        for target in targets:
            for word in target:
                if word in target_dicts:
                    target_dicts[word] += 1
                else:
                    target_dicts[word] = 1
        for dict in target_dicts:
            if target_dicts[dict] >=2:
                final_targets.append(dict)
        
        final_data = {}
        final_data["post_tokens"] = data["post_tokens"]
        final_data["label"] = final_label
        final_data["target"] = final_targets
        easy_data.append(final_data)
    return easy_data

In [22]:
#Processing train data
easy_train_data = process_easy_data(train_data)
print("easy_train_data length: ", len(easy_train_data))
print(easy_train_data[:10])
# Save list as a JSON file
with open('easy_train_data.json', 'w') as f:
    json.dump(easy_train_data, f)

easy_train_data length:  15383
[{'post_tokens': ['u', 'really', 'think', 'i', 'would', 'not', 'have', 'been', 'raped', 'by', 'feral', 'hindu', 'or', 'muslim', 'back', 'in', 'india', 'or', 'bangladesh', 'and', 'a', 'neo', 'nazi', 'would', 'rape', 'me', 'as', 'well', 'just', 'to', 'see', 'me', 'cry'], 'label': 2, 'target': ['Hindu', 'Islam']}, {'post_tokens': ['the', 'uk', 'has', 'threatened', 'to', 'return', 'radioactive', 'waste', 'to', 'the', 'eu', 'if', 'an', 'agreement', 'cannot', 'be', 'reached', 'can', 'not', 'we', 'keep', 'the', 'radioactive', 'waste', 'and', 'send', 'back', 'all', 'the', 'paki', 'migrants', 'instead'], 'label': 2, 'target': ['Refugee', 'Indian']}, {'post_tokens': ['if', 'english', 'is', 'not', 'imposition', 'then', 'hindi', 'is', 'also', 'not', 'imposition', 'shut', 'up', 'chutiya', 'retards', 'stophindiimposition'], 'label': 2, 'target': ['Other', 'Hindu']}, {'post_tokens': ['no', 'liberal', 'congratulated', 'hindu', 'refugees', 'post', 'cab', 'because', 'they'

In [8]:
#Process eval data
easy_eval_data = process_easy_data(val_data)
print("easy_eval_data length: ", len(easy_eval_data))
print(easy_eval_data[:10])

# Save list as a JSON file
with open('easy_eval_data.json', 'w') as f:
    json.dump(easy_eval_data, f)

easy_eval_data length:  1922
[{'post_tokens': ['me', 'getting', 'books', 'from', 'the', 'library', 'about', 'queer', 'cowboys', 'and', 'homoeroticism', 'in', 'cinema', 'instead', 'of', 'anything', 'useful', 'for', 'my', 'thesis'], 'label': 1, 'target': ['None']}, {'post_tokens': ['y', 'si', 'fuera', 'top', '<number>', 'me', 'faltarían', 'how', 'to', 'get', 'away', 'with', 'murder', 'gossip', 'girl', 'the', 'last', 'ship', 'orphan', 'black', 'downton', 'abbey'], 'label': 1, 'target': ['None']}, {'post_tokens': ['<user>', '<user>', '<user>', 'i', 'am', 'a', 'lesbian', 'not', 'queer', 'and', 'quite', 'frankly', 'his', 'disco', 'tits', 'video', 'was', 'repulsive', 'if', 'he', 'thinks', 'being', 'an', 'overweight', 'gay', 'bloke', 'brings', 'out', 'the', 'woman', 'in', 'him', 'then', 'i', 'am', 'offended', 'it', 'pathetic', 'attention', 'seeking'], 'label': 1, 'target': ['None']}, {'post_tokens': ['<user>', 'by', 'tweeting', 'about', 'a', 'civil', 'war', 'trump', 'is', 'sending', 'a', 'mess